# Day 14: Final Preparation
Perform sensitivity analysis and prepare a stakeholder presentation for the capstone project.

In [ ]:
from mosek.fusion import *
import numpy as np
import pandas as pd

def sensitivity_analysis(returns: pd.DataFrame, var_limit: float, lcr_limit: float, hqla_factors: np.ndarray, outflows: float, vol_scale: float = 1.0) -> tuple:
    scaled_returns = returns * vol_scale
    n = returns.shape[1]
    with Model('Sensitivity') as M:
        m = M.variable('m', n, Domain.greaterThan(0.25))
        portfolio_returns = Expr.dot(m, scaled_returns.T)
        M.constraint('var', Expr.mul(-1.645, Expr.stdDev(portfolio_returns)), Domain.lessThan(var_limit))
        M.constraint('lcr', Expr.dot(hqla_factors, m), Domain.greaterThan(lcr_limit * outflows))
        M.objective('obj', ObjectiveSense.Minimize, Expr.sum(m))
        M.solve()
        margins = m.level()
        total_margin = np.sum(margins)
    return margins, total_margin

np.random.seed(42)
data = pd.DataFrame({
    'Equity': np.random.normal(0, 0.01, 252),
    'Credit': np.random.normal(0, 0.015, 252),
    'Commodity': np.random.normal(0, 0.02, 252)
})
hqla_factors = np.array([0.8, 0.9, 0.6])
scales = [1.0, 1.1, 1.2]
results = []
for scale in scales:
    margins, total = sensitivity_analysis(data, 0.05, 1.0, hqla_factors, 10.0, scale)
    results.append(total)
print(f'Sensitivity Results (Total Margin): {dict(zip(scales, results))}')

## Presentation Script: Capstone Project

**Slide 1: Introduction (1 minute)**
- Hello, team. I’m presenting our capstone project: a multi-asset margin allocation model for Equities, Credit, and Commodities.
- Objective: Minimize margins while ensuring VaR ≤ 5%, LCR ≥ 1, and FINRA compliance.

**Slide 2: Methodology (2 minutes)**
- **Model**: Quadratic optimization using Mosek.
- **Constraints**:
  - VaR ≤ 0.05 (95% confidence, z-score = 1.645).
  - LCR ≥ 1 using HQLA factors.
  - Minimum margin 25% per asset (FINRA).
- **Attribution**: Simplified Shapley values for margin contributions.
- **Data**: 252 days of simulated returns.

**Slide 3: Results (3 minutes)**
- **Margins**: Optimized allocations for Equity, Credit, Commodity.
- **Attribution**: Commodity drives margins due to high volatility (see bar chart).
- **Sensitivity**: 10% volatility increase raises total margin by 8%.
- **Scenarios**: High-correlation stress scenario increases VaR by 12%.

**Slide 4: Business Impact (2 minutes)**
- **Capital Efficiency**: Cross-margining reduces capital requirements.
- **Compliance**: Meets FINRA and Basel III standards.
- **Scalability**: Modular code supports additional assets.
- **Transparency**: Attribution aids PMs and Risk in decision-making.

**Slide 5: Next Steps (1 minute)**
- Deploy model with IT in 4 weeks.
- Enhance with stochastic scenarios and full Shapley attribution.
- Schedule follow-up reviews with stakeholders.

**Slide 6: Q&A (1 minute)**
- Questions? (Prepared for: black swan events, data quality, regulatory changes)

**Total Time**: 10 minutes